In [ ]:
# Resumen final del análisis
cat("=== RESUMEN FINAL DEL ANÁLISIS TWEEDIE ===\n")
cat("==========================================\n")

cat("📊 DATOS:\n")
cat("   - Observaciones analizadas:", nrow(datos_modelo), "\n")
cat("   - Variable respuesta: SumaDePagos (transformada a y_tweedie)\n")
cat("   - Variables predictoras disponibles:", length(vars_predictoras), "\n")

cat("\n🔧 PROCESAMIENTO:\n")
cat("   - Variables categóricas tratadas con as.character() y relevel()\n")
cat("   - Variables numéricas winzorizadas (P1-P99)\n")
cat("   - Ceros reemplazados por valor:", valor_remplazo, "\n")

cat("\n📈 MODELO SELECCIONADO:\n")
cat("   - Familia: Tweedie con parámetro p =", p_optimo, "\n")
cat("   - Variables en modelo final:", length(variables_incluidas), "\n")
if(length(variables_incluidas) > 0) {
  cat("   - Variables incluidas:", paste(variables_incluidas, collapse = ", "), "\n")
}
cat("   - AIC final:", round(AIC(modelo_final), 2), "\n")
cat("   - Pseudo R² (McFadden):", round(pseudo_r2, 4), "\n")
cat("   - Convergencia:", if(modelo_final$converged) "✅ Sí" else "❌ No", "\n")

cat("\n🎯 DIAGNÓSTICO (ENVELOPE):\n")
cat("   - Residuos analizados:", n_residuos, "\n")
cat("   - Puntos fuera del envelope:", puntos_fuera, "(", round(prop_fuera * 100, 1), "%)\n")
cat("   - Calidad del ajuste:", 
    if(prop_fuera <= 0.05) "✅ Excelente" 
    else if(prop_fuera <= 0.10) "✅ Bueno" 
    else if(prop_fuera <= 0.15) "⚠️ Moderado" 
    else "❌ Problemático", "\n")

cat("\n💡 INTERPRETACIÓN ACTUARIAL:\n")
if(p_optimo < 1.4) {
  cat("   - Comportamiento Poisson-like: muchos ceros (no siniestros)\n")
  cat("   - Apropiado para modelar frecuencia de siniestros\n")
} else if(p_optimo > 1.6) {
  cat("   - Comportamiento Gamma-like: pocos ceros, alta variabilidad\n")
  cat("   - Apropiado para modelar severidad de siniestros\n")
} else {
  cat("   - Balance Poisson-Gamma: típico para modelos de siniestralidad\n")
  cat("   - Modelo apropiado para pricing de seguros\n")
}

cat("\n✅ ANÁLISIS COMPLETADO EXITOSAMENTE ✅\n")

In [ ]:
# Visualización del Envelope Plot
envelope_plot <- ggplot(df_envelope) +
  # Banda del envelope
  geom_ribbon(aes(x = teoricos, ymin = lower, ymax = upper), 
              fill = "lightblue", alpha = 0.6) +
  # Puntos observados
  geom_point(aes(x = teoricos, y = observados), 
             size = 1.5, alpha = 0.7, color = "darkblue") +
  # Línea de referencia (y = x)
  geom_abline(intercept = 0, slope = 1, color = "red", linewidth = 1, linetype = "dashed") +
  # Etiquetas y tema
  labs(
    title = "Envelope Plot - Diagnóstico del Modelo Tweedie",
    subtitle = paste0("Parámetro p = ", p_optimo, " | ", 
                     puntos_fuera, " puntos fuera del envelope (",
                     round(prop_fuera * 100, 1), "%)"),
    x = "Cuantiles Teóricos (Normal Estándar)",
    y = if(usar_quantile) "Residuos Cuantílicos" else "Residuos de Pearson",
    caption = paste0("Bandas de confianza al 95% basadas en ", n_simulaciones, " simulaciones")
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(hjust = 0.5, size = 14, face = "bold"),
    plot.subtitle = element_text(hjust = 0.5, size = 12),
    plot.caption = element_text(hjust = 0.5, size = 10, color = "gray50")
  )

# Mostrar el plot
print(envelope_plot)

# Interpretación estadística
cat("\n=== INTERPRETACIÓN DEL ENVELOPE PLOT ===\n")

if(prop_fuera <= 0.05) {
  cat("✅ EXCELENTE AJUSTE: ≤ 5% de puntos fuera del envelope\n")
  cat("   El modelo se ajusta muy bien a los datos\n")
} else if(prop_fuera <= 0.10) {
  cat("✅ BUEN AJUSTE: ≤ 10% de puntos fuera del envelope\n")
  cat("   El modelo se ajusta adecuadamente a los datos\n")
} else if(prop_fuera <= 0.15) {
  cat("⚠️  AJUSTE MODERADO: 10-15% de puntos fuera del envelope\n")
  cat("   El modelo tiene algunas deficiencias menores\n")
} else {
  cat("❌ AJUSTE PROBLEMÁTICO: > 15% de puntos fuera del envelope\n")
  cat("   El modelo no se ajusta bien a los datos\n")
}

cat("\nInterpretación adicional:\n")
cat("- Puntos por encima del envelope: posibles outliers o varianza subestimada\n")
cat("- Puntos por debajo del envelope: posible sobre-dispersión no modelada\n")
cat("- Patrones sistemáticos: violaciones de supuestos del modelo\n")

In [ ]:
# Construcción del Envelope Plot
cat("=== CONSTRUCCIÓN DEL ENVELOPE PLOT ===\n")

if(n_residuos < 20) {
  cat("ADVERTENCIA: Muy pocos residuos válidos para un envelope confiable\n")
}

# Ordenar residuos
residuos_ordenados <- sort(residuos_limpios)

# Cuantiles teóricos de una distribución normal estándar
cuantiles_teoricos <- qnorm(ppoints(n_residuos))

cat("Construyendo envelope mediante simulación...\n")

# Parámetros para la simulación del envelope
n_simulaciones <- 99  # Número de simulaciones para construir envelope
alpha <- 0.05         # Nivel de significancia (bandas al 95%)

# Generar simulaciones para el envelope
set.seed(123)  # Para reproducibilidad
simulaciones <- matrix(NA, nrow = n_simulaciones, ncol = n_residuos)

for(i in 1:n_simulaciones) {
  # Simular residuos normales y ordenarlos
  sim_residuos <- rnorm(n_residuos)
  simulaciones[i, ] <- sort(sim_residuos)
}

# Calcular bandas del envelope
envelope_lower <- apply(simulaciones, 2, quantile, alpha/2)
envelope_upper <- apply(simulaciones, 2, quantile, 1 - alpha/2)

cat("Envelope construido con", n_simulaciones, "simulaciones\n")
cat("Bandas de confianza al", (1-alpha)*100, "%\n")

# Crear data frame para el gráfico
df_envelope <- data.frame(
  teoricos = cuantiles_teoricos,
  observados = residuos_ordenados,
  lower = envelope_lower,
  upper = envelope_upper
)

# Contar puntos fuera del envelope
puntos_fuera <- sum(df_envelope$observados < df_envelope$lower | 
                   df_envelope$observados > df_envelope$upper)
prop_fuera <- puntos_fuera / n_residuos

cat("Puntos fuera del envelope:", puntos_fuera, "de", n_residuos, 
    "(", round(prop_fuera * 100, 2), "%)\n")

In [ ]:
residuos_pearson <- residuals(modelo_final, type = "pearson")
residuos_deviance <- residuals(modelo_final, type = "deviance")

summary(residuos_pearson)
residuos_limpios <- residuos_pearson[is.finite(residuos_pearson) & !is.na(residuos_pearson)]
n_residuos <- length(residuos_limpios)

## Diagnóstico del Modelo

In [ ]:
# Modelo final seleccionado
cat("=== MODELO FINAL SELECCIONADO ===\n")

modelo_final <- modelo_actual
cat("Fórmula final:", deparse(formula(modelo_final)), "\n")
cat("Parámetro p usado:", p_optimo, "\n")
cat("Convergió:", modelo_final$converged, "\n")
cat("AIC final:", round(AIC(modelo_final), 2), "\n")
cat("Deviance:", round(deviance(modelo_final), 2), "\n")
cat("Número de parámetros:", length(coef(modelo_final)), "\n")
cat("Observaciones:", nobs(modelo_final), "\n")

# Pseudo R² (McFadden)
pseudo_r2 <- 1 - (deviance(modelo_final) / deviance(modelo_nulo))
cat("Pseudo R² (McFadden):", round(pseudo_r2, 4), "\n")

cat("\n=== RESUMEN DEL MODELO ===\n")
summary(modelo_final)

In [ ]:
# Selección de modelo mediante AIC Forward
cat("\n=== SELECCIÓN DE MODELO AIC FORWARD ===\n")

# Definir familia Tweedie con p óptimo
familia_tweedie <- tweedie(var.power = p_optimo, link.power = 0)

# Modelo nulo (solo intercepto)
modelo_nulo <- glm(y_tweedie ~ 1, 
                  data = datos_modelo, 
                  family = familia_tweedie)

cat("Modelo inicial (solo intercepto):\n")
cat("AIC inicial:", round(AIC(modelo_nulo), 2), "\n")
cat("Deviance inicial:", round(deviance(modelo_nulo), 2), "\n\n")

# Algoritmo Forward Selection manual
modelo_actual <- modelo_nulo
aic_actual <- AIC(modelo_actual)
variables_incluidas <- character(0)
variables_disponibles <- vars_predictoras

# Resultado del proceso forward
historial_forward <- data.frame(
  Paso = integer(0),
  Variable_Agregada = character(0),
  AIC = numeric(0),
  Delta_AIC = numeric(0),
  stringsAsFactors = FALSE
)

paso <- 0

cat("=== PROCESO FORWARD STEP-BY-STEP ===\n")

while(length(variables_disponibles) > 0) {
  paso <- paso + 1
  
  cat("\n--- PASO", paso, "---\n")
  cat("Variables disponibles:", paste(variables_disponibles, collapse = ", "), "\n")
  
  mejor_aic <- Inf
  mejor_variable <- NULL
  
  # Probar agregar cada variable disponible
  for(var in variables_disponibles) {
    
    # Crear fórmula con variables actuales + nueva variable
    if(length(variables_incluidas) == 0) {
      formula_test <- as.formula(paste("y_tweedie ~", var))
    } else {
      formula_test <- as.formula(paste("y_tweedie ~", 
                                      paste(variables_incluidas, collapse = " + "), 
                                      "+", var))
    }
    
    tryCatch({
      modelo_test <- glm(formula_test, 
                        data = datos_modelo, 
                        family = familia_tweedie)
      
      if(modelo_test$converged) {
        aic_test <- AIC(modelo_test)
        cat("  ", var, "-> AIC =", round(aic_test, 2), "\n")
        
        if(aic_test < mejor_aic) {
          mejor_aic <- aic_test
          mejor_variable <- var
        }
      } else {
        cat("  ", var, "-> No convergió\n")
      }
      
    }, error = function(e) {
      cat("  ", var, "-> Error:", e$message, "\n")
    })
  }
  
  # Verificar si hay mejora
  if(!is.null(mejor_variable) && mejor_aic < aic_actual) {
    
    # Actualizar modelo
    variables_incluidas <- c(variables_incluidas, mejor_variable)
    variables_disponibles <- variables_disponibles[variables_disponibles != mejor_variable]
    
    # Ajustar modelo actualizado
    if(length(variables_incluidas) == 1) {
      formula_nueva <- as.formula(paste("y_tweedie ~", variables_incluidas[1]))
    } else {
      formula_nueva <- as.formula(paste("y_tweedie ~", paste(variables_incluidas, collapse = " + ")))
    }
    
    modelo_actual <- glm(formula_nueva, data = datos_modelo, family = familia_tweedie)
    delta_aic <- mejor_aic - aic_actual
    aic_actual <- mejor_aic
    
    # Registrar en historial
    historial_forward <- rbind(historial_forward, 
                              data.frame(Paso = paso,
                                        Variable_Agregada = mejor_variable,
                                        AIC = mejor_aic,
                                        Delta_AIC = delta_aic,
                                        stringsAsFactors = FALSE))
    
    cat("\n✓ AGREGADA:", mejor_variable, "\n")
    cat("  Nuevo AIC:", round(mejor_aic, 2), "\n")
    cat("  Mejora AIC:", round(delta_aic, 2), "\n")
    cat("  Variables en modelo:", paste(variables_incluidas, collapse = ", "), "\n")
    
  } else {
    cat("\n✗ No hay mejora significativa. Deteniendo forward selection.\n")
    break
  }
}

cat("\n=== RESULTADO FINAL FORWARD SELECTION ===\n")
cat("Variables seleccionadas:", paste(variables_incluidas, collapse = ", "), "\n")
cat("AIC final:", round(aic_actual, 2), "\n")
cat("Mejora total AIC:", round(AIC(modelo_nulo) - aic_actual, 2), "\n")

print(historial_forward)

In [ ]:
p_candidatos <- seq(1.1, 1.9, by = 0.1)
aic_resultados <- numeric(length(p_candidatos))

vars_predictoras <- c("Color", "Carroceria", "CLASE_FASECOLDA", 
                     "TIPO_VEHICULO", "SERVICIO", "Sexo_Aseg", 
                     "Edad", "Vr_Comercial", "Accidentado")

for(i in seq_along(p_candidatos)) {
  p_test <- p_candidatos[i]
  
  tryCatch({
    modelo_test <- glm(y_tweedie ~ 1, 
                      data = datos_modelo,
                      family = tweedie(var.power = p_test, link.power = 0))
    
    if(modelo_test$converged) {
      aic_resultados[i] <- AIC(modelo_test)
    } else {
      aic_resultados[i] <- Inf
    }
    
  }, error = function(e) {
    aic_resultados[i] <- Inf
  })
}

p_optimo <- p_candidatos[which.min(aic_resultados)]
cat("Parámetro p óptimo:", p_optimo, "\n")

## Selección de Modelo

In [ ]:
# Tratamiento de variables numéricas
cat("\n=== TRATAMIENTO DE VARIABLES NUMÉRICAS ===\n")

vars_numericas <- c("Edad", "Vr_Comercial")

for(var in vars_numericas) {
  if(var %in% names(datos_modelo)) {
    cat("\n--- Procesando variable numérica:", var, "---\n")
    
    x <- datos_modelo[[var]]
    
    # Tratar valores faltantes con la mediana
    n_na <- sum(is.na(x))
    if(n_na > 0) {
      mediana <- median(x, na.rm = TRUE)
      x[is.na(x)] <- mediana
      cat("Reemplazados", n_na, "valores faltantes con mediana =", mediana, "\n")
    }
    
    # Winsorización de outliers extremos (P1 y P99)
    p01 <- quantile(x, 0.01, na.rm = TRUE)
    p99 <- quantile(x, 0.99, na.rm = TRUE)
    
    outliers_inf <- sum(x < p01)
    outliers_sup <- sum(x > p99)
    
    if(outliers_inf > 0) {
      x[x < p01] <- p01
      cat("Winsorized", outliers_inf, "outliers inferiores al P1 =", p01, "\n")
    }
    
    if(outliers_sup > 0) {
      x[x > p99] <- p99
      cat("Winsorized", outliers_sup, "outliers superiores al P99 =", p99, "\n")
    }
    
    # Actualizar en el dataset
    datos_modelo[[var]] <- x
    
    cat("Rango final:", range(x), "\n")
  }
}

# Mostrar resumen del dataset preparado
cat("\n=== RESUMEN DEL DATASET PREPARADO ===\n")
cat("Dimensiones finales:", dim(datos_modelo), "\n")
cat("Variables categóricas procesadas:", length(vars_categoricas), "\n")
cat("Variables numéricas procesadas:", length(vars_numericas), "\n")

In [ ]:
# Tratamiento explícito de variables categóricas usando as.character() y relevel()
cat("\n=== TRATAMIENTO DE VARIABLES CATEGÓRICAS ===\n")

# Lista de variables categóricas a procesar
vars_categoricas <- c("Color", "Carroceria", "CLASE_FASECOLDA", 
                     "TIPO_VEHICULO", "SERVICIO", "Sexo_Aseg", "Accidentado")

for(var in vars_categoricas) {
  cat("\n--- Procesando variable:", var, "---\n")
  
  # Paso 1: Convertir explícitamente a character
  var_char <- as.character(datos_modelo[[var]])
  cat("Convertida a character, clase:", class(var_char), "\n")
  
  # Paso 2: Limpiar valores problemáticos
  # Reemplazar NAs, espacios vacíos, etc.
  var_char[is.na(var_char) | var_char == "" | var_char == " "] <- "Desconocido"
  
  # Paso 3: Analizar frecuencias y agrupar niveles raros
  tabla_freq <- table(var_char)
  n_niveles_inicial <- length(tabla_freq)
  
  # Agrupar niveles con menos de 10 observaciones en "Otros"
  niveles_raros <- names(tabla_freq)[tabla_freq < 10]
  if(length(niveles_raros) > 0) {
    var_char[var_char %in% niveles_raros] <- "Otros"
    cat("Agrupados", length(niveles_raros), "niveles raros en 'Otros'\n")
  }
  
  # Paso 4: Convertir a factor
  var_factor <- factor(var_char)
  n_niveles_final <- nlevels(var_factor)
  
  cat("Niveles: inicial =", n_niveles_inicial, ", final =", n_niveles_final, "\n")
  
  # Paso 5: Aplicar relevel() explícitamente con as.character()
  # Usar el nivel más frecuente como referencia
  tabla_final <- table(var_factor)
  nivel_mas_frecuente <- names(tabla_final)[which.max(tabla_final)]
  
  cat("Nivel más frecuente:", nivel_mas_frecuente, 
      "(", max(tabla_final), "observaciones)\n")
  
  # Aplicar relevel usando as.character explícitamente
  var_factor_releveled <- relevel(var_factor, ref = as.character(nivel_mas_frecuente))
  
  cat("Nivel de referencia establecido:", levels(var_factor_releveled)[1], "\n")
  cat("Todos los niveles:", paste(levels(var_factor_releveled), collapse = ", "), "\n")
  
  # Asignar la variable procesada al dataset
  datos_modelo[[var]] <- var_factor_releveled
  
  # Mostrar tabla final de frecuencias
  cat("Tabla final de frecuencias:\n")
  print(table(datos_modelo[[var]]))
}

In [ ]:
datos_modelo <- datos
min_positivo <- min(datos_modelo$SumaDePagos[datos_modelo$SumaDePagos > 0])
valor_remplazo <- min_positivo / 1000

datos_modelo$y_tweedie <- ifelse(datos_modelo$SumaDePagos == 0, 
                                valor_remplazo, 
                                datos_modelo$SumaDePagos)

## Preparación de Datos

In [ ]:
# Análisis de variables numéricas
cat("=== ANÁLISIS DE VARIABLES NUMÉRICAS ===\n")

variables_numericas <- c("Edad", "Vr_Comercial", "exposicion")

for(var in variables_numericas) {
  if(var %in% names(datos)) {
    cat("\n--- Variable:", var, "---\n")
    
    x <- datos[[var]]
    
    # Estadísticas descriptivas
    cat("Estadísticas básicas:\n")
    print(summary(x))
    
    # Detectar valores faltantes
    n_na <- sum(is.na(x))
    if(n_na > 0) {
      cat("Valores faltantes:", n_na, "(", round(n_na/length(x) * 100, 2), "%)\n")
    }
    
    # Detectar outliers usando criterio IQR
    q1 <- quantile(x, 0.25, na.rm = TRUE)
    q3 <- quantile(x, 0.75, na.rm = TRUE)
    iqr <- q3 - q1
    outliers_inf <- sum(x < (q1 - 1.5 * iqr), na.rm = TRUE)
    outliers_sup <- sum(x > (q3 + 1.5 * iqr), na.rm = TRUE)
    
    cat("Outliers inferiores:", outliers_inf, "\n")
    cat("Outliers superiores:", outliers_sup, "\n")
    cat("Total outliers:", outliers_inf + outliers_sup, 
        "(", round((outliers_inf + outliers_sup)/length(x) * 100, 2), "%)\n")
  }
}

In [ ]:
# Análisis de variables categóricas
cat("=== ANÁLISIS DE VARIABLES CATEGÓRICAS ===\n")

variables_categoricas <- c("Amparo", "Color", "Carroceria", "CLASE_FASECOLDA", 
                          "TIPO_VEHICULO", "SERVICIO", "Sexo_Aseg", "Accidentado")

for(var in variables_categoricas) {
  if(var %in% names(datos)) {
    cat("\n--- Variable:", var, "---\n")
    
    # Convertir a character para análisis
    var_char <- as.character(datos[[var]])
    
    # Tabla de frecuencias
    tabla_freq <- table(var_char, useNA = "ifany")
    n_niveles <- length(tabla_freq)
    
    cat("Número de niveles únicos:", n_niveles, "\n")
    cat("Tabla de frecuencias:\n")
    print(sort(tabla_freq, decreasing = TRUE))
    
    # Calcular proporción de la moda
    freq_max <- max(tabla_freq)
    prop_moda <- freq_max / sum(tabla_freq)
    cat("Proporción de la moda:", round(prop_moda * 100, 2), "%\n")
  }
}

In [ ]:
# Visualizaciones de la variable respuesta
p1 <- ggplot(datos, aes(x = SumaDePagos)) +
  geom_histogram(bins = 100, fill = "lightblue", alpha = 0.7) +
  labs(title = "Distribución de SumaDePagos (Todos los valores)",
       x = "Suma de Pagos", y = "Frecuencia") +
  theme_minimal()

# Solo valores positivos en escala logarítmica
datos_pos <- datos[datos$SumaDePagos > 0, ]
p2 <- ggplot(datos_pos, aes(x = SumaDePagos)) +
  geom_histogram(bins = 50, fill = "steelblue", alpha = 0.7) +
  scale_x_log10() +
  labs(title = "Distribución de SumaDePagos (Valores positivos - log10)",
       x = "Suma de Pagos (log10)", y = "Frecuencia") +
  theme_minimal()

# Mostrar ambos gráficos
grid.arrange(p1, p2, ncol = 2)

In [ ]:
summary(datos$SumaDePagos)
n_total <- length(datos$SumaDePagos)
n_ceros <- sum(datos$SumaDePagos == 0)
prop_ceros <- n_ceros / n_total

cat("Total observaciones:", n_total, "\n")
cat("Valores cero:", n_ceros, "(", round(prop_ceros * 100, 2), "%)\n")

## Análisis Descriptivo

In [ ]:
# Mostrar primeras observaciones
cat("=== PRIMERAS 10 OBSERVACIONES ===\n")
head(datos, 10)

# Resumen de todas las variables
cat("\n=== RESUMEN ESTADÍSTICO INICIAL ===\n")
summary(datos)

In [ ]:
ruta_datos <- "../data/processed/datos_limpios.csv"
datos <- read.csv(ruta_datos, stringsAsFactors = FALSE)
str(datos)

## Carga de Datos

In [ ]:
# Cargar librerías necesarias
library(statmod)
library(tweedie)
library(ggplot2)
library(gridExtra)
library(corrplot)

# Configuración de opciones
options(scipen = 999)  # Evitar notación científica
set.seed(123)          # Reproducibilidad

# Análisis de Modelo Tweedie para Datos de Seguros